In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
import rioxarray as rxr
import xarray as xr
import folium
import hvplot
import h5py
import h5netcdf
import re

In [ ]:
# grap data from hdf5 file
f = h5py.File("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/VIIRS_SCA/2013-2014/VNP10A1F.A2013330.h08v05.001.2019340224942.h5", "r")

In [ ]:
# # List all groups
# print("Keys: %s" % f.keys())
# a_group_key = list(f.keys())[0]
#
# # Get the data
# data = list(f[a_group_key])

In [ ]:
for key in f.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key])) # get the object type: usually group or dataset
    ls = list(f.keys())

In [ ]:
# np.array(f.get('HDFEOS').get('GRIDS').get('NPP_Grid_IMG_2D').get('Data Fields').get('CGF_NDSI_Snow_Cover'))
# bitmap = np.array(f['HDFEOS']['GRIDS']['NPP_Grid_IMG_2D']['Data Fields']['CGF_NDSI_Snow_Cover'])
# bitmap

In [ ]:
# xr.open_mfdataset("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/VIIRS_SCA/2013-2014/VNP10A1F.A2013330.h08v05.001.2019340224942.h5", engine='h5netcdf')

In [ ]:
# def create_geotiff(fname, h5_path, geo_transform, projection = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs", datafield = "CGF_NDSI_Snow_Cover"):
#     '''
#     Create a GeoTIFF file with the given data.
#     '''
#     # Open the HDF5 file
#     hdf5_ds = h5py.File(h5_path, 'r')
#     data = hdf5_ds['HDFEOS']['GRIDS']['NPP_Grid_IMG_2D']['Data Fields'][datafield]
#     crs = rio.crs.CRS.from_proj4(projection)



In [ ]:
def convert_h5_to_geotiff(h5_file, datafield_name, output_geotiff):
    with h5py.File(h5_file, mode='r') as f:
        # Access the data field
        data = f[datafield_name][:].astype(np.float64)

        # Read metadata
        gridmeta = f['/HDFEOS INFORMATION/StructMetadata.0'][()]
        s = gridmeta.decode('UTF-8')

        # Extract grid extents using regular expressions
        ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
        (?P<upper_left_x>[+-]?\d+\.\d+)
        ,
        (?P<upper_left_y>[+-]?\d+\.\d+)
        \)''', re.VERBOSE)
        match = ul_regex.search(s)
        x0 = float(match.group('upper_left_x'))
        y0 = float(match.group('upper_left_y'))
        lr_regex = re.compile(r'''LowerRightMtrs=\(
        (?P<lower_right_x>[+-]?\d+\.\d+)
        ,
        (?P<lower_right_y>[+-]?\d+\.\d+)
        \)''', re.VERBOSE)
        match = lr_regex.search(s)
        x1 = float(match.group('lower_right_x'))
        y1 = float(match.group('lower_right_y'))
        ny, nx = data.shape

        # Create rasterio dataset and write the data
        transform = rio.transform.from_bounds(x0, y1, x1, y0, nx, ny)
        # crs = rio.crs.CRS.from_proj4("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
        # crs = rio.crs.CRS.from_wkt('''
        #     PROJCS["MODIS Sinusoidal",
        #         GEOGCS["WGS 84",
        #             DATUM["WGS_1984",
        #                 SPHEROID["WGS 84",6378137,298.257223563,
        #                     AUTHORITY["EPSG","7030"]],
        #                 AUTHORITY["EPSG","6326"]],
        #             PRIMEM["Greenwich",0,
        #                 AUTHORITY["EPSG","8901"]],
        #             UNIT["degree",0.01745329251994328,
        #                 AUTHORITY["EPSG","9122"]],
        #             AUTHORITY["EPSG","4326"]],
        #         PROJECTION["Sinusoidal"],
        #         PARAMETER["false_easting",0.0],
        #         PARAMETER["false_northing",0.0],
        #         PARAMETER["central_meridian",0.0],
        #         PARAMETER["semi_major",6371007.181],
        #         PARAMETER["semi_minor",6371007.181],
        #         UNIT["m",1.0],
        #         AUTHORITY["SR-ORG","6974"]]''')
        crs = rio.crs.CRS.from_wkt(
            '''
            PROJCRS["Sinusoidal Modis Spheroid",BASEGEOGCRS["Unknown datum based upon the Authalic Sphere",DATUM["Not specified (based on Authalic Sphere)",ELLIPSOID["Sphere",6371007.181,0,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4035]],CONVERSION["Sinusoidal",METHOD["Sinusoidal"],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]
            '''
        )
        with rio.open(output_geotiff, 'w', driver='GTiff', height=ny, width=nx, count=1, dtype=data.dtype,
                           crs=crs, transform=transform) as dst:
                        dst.write(data, 1)

In [ ]:
def convert_h5_to_geotiff(h5_file, datafield_name, output_geotiff):
    with h5py.File(h5_file, mode='r') as f:
        # Access the data field
        data = f[datafield_name][:].astype(np.float64)

        # Read metadata
        gridmeta = f['/HDFEOS INFORMATION/StructMetadata.0'][()]
        s = gridmeta.decode('UTF-8')

        # Extract grid extents using regular expressions
        ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
        (?P<upper_left_x>[+-]?\d+\.\d+)
        ,
        (?P<upper_left_y>[+-]?\d+\.\d+)
        \)''', re.VERBOSE)
        match = ul_regex.search(s)
        x0 = float(match.group('upper_left_x'))
        y0 = float(match.group('upper_left_y'))
        lr_regex = re.compile(r'''LowerRightMtrs=\(
        (?P<lower_right_x>[+-]?\d+\.\d+)
        ,
        (?P<lower_right_y>[+-]?\d+\.\d+)
        \)''', re.VERBOSE)
        match = lr_regex.search(s)
        x1 = float(match.group('lower_right_x'))
        y1 = float(match.group('lower_right_y'))
        ny, nx = data.shape

        # Create rasterio dataset and write the data
        transform = rio.transform.from_bounds(x0, y1, x1, y0, nx, ny)
        # crs = rio.crs.CRS.from_proj4("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
        # crs = rio.crs.CRS.from_wkt('''
        #     PROJCS["MODIS Sinusoidal",
        #         GEOGCS["WGS 84",
        #             DATUM["WGS_1984",
        #                 SPHEROID["WGS 84",6378137,298.257223563,
        #                     AUTHORITY["EPSG","7030"]],
        #                 AUTHORITY["EPSG","6326"]],
        #             PRIMEM["Greenwich",0,
        #                 AUTHORITY["EPSG","8901"]],
        #             UNIT["degree",0.01745329251994328,
        #                 AUTHORITY["EPSG","9122"]],
        #             AUTHORITY["EPSG","4326"]],
        #         PROJECTION["Sinusoidal"],
        #         PARAMETER["false_easting",0.0],
        #         PARAMETER["false_northing",0.0],
        #         PARAMETER["central_meridian",0.0],
        #         PARAMETER["semi_major",6371007.181],
        #         PARAMETER["semi_minor",6371007.181],
        #         UNIT["m",1.0],
        #         AUTHORITY["SR-ORG","6974"]]''')
        crs = rio.crs.CRS.from_wkt(
            '''
            PROJCRS["Sinusoidal Modis Spheroid",BASEGEOGCRS["Unknown datum based upon the Authalic Sphere",DATUM["Not specified (based on Authalic Sphere)",ELLIPSOID["Sphere",6371007.181,0,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4035]],CONVERSION["Sinusoidal",METHOD["Sinusoidal"],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]
            '''
        )
        with rio.open(output_geotiff, 'w', driver='GTiff', height=ny, width=nx, count=1, dtype=data.dtype,
                           crs=crs, transform=transform) as dst:
                        dst.write(data, 1)

In [ ]:
fpath = "/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/VIIRS_SCA/2013-2014/VNP10A1F.A2013365.h09v04.001.2019341004913.h5"

datafield = "/HDFEOS/GRIDS/NPP_Grid_IMG_2D/Data Fields/CGF_NDSI_Snow_Cover/"

convert_h5_to_geotiff(fpath, datafield, "testh09v04_stolen_ints.tif")

In [ ]:
fpath = "/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/VIIRS_SCA/VNP10A1.A2013365.h12v12.001.2019211062842.h5"
datafield = "/HDFEOS/GRIDS/NPP_Grid_IMG_2D/Data Fields/CGF_NDSI_Snow_Cover/"

# convert_h5_to_geotiff(fpath, datafield, "testh12v12_stolenCRS.tif")

In [ ]:
# plot geotiff
# Open the file:
gran1 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/testh08v04_stolen.tif", 'r')
gran2 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/testh09v04_stolen.tif", 'r')
gran3 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/testh08v05_stolen.tif", 'r')

In [ ]:
import rioxarray as rxr
from rioxarray.merge import merge_arrays
def createMergedRxr(files: list[str]):
    # Create a merged rasterio dataset from a list of files
    tifs = [rxr.open_rasterio(file) for file in files]
    merged = merge_arrays(tifs, nodata=255)
    return merged

In [ ]:
l = [
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h07v05_002_2023042125033_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe7c713.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h08v04_002_2023042130237_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe7dd78.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h08v05_002_2023042125025_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe7ceb1.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h09v04_002_2023042130201_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe7d655.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h09v05_002_2023042123953_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe76cd7.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h10v04_002_2023042132519_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe8478d.tif",
"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/outputs/VNP10A1F_A2020193_h11v04_002_2023042131558_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_efe85653.tif",
]

In [ ]:
merged = createMergedRxr(l)

In [ ]:
merged

In [ ]:
image = merged.values
show(image)

In [ ]:
merged.rio.crs

In [ ]:

merged, transform = merge([gran1, gran2, gran3])
# hvplot.plot(merged, kind="", cmap='viridis', clim=(0, 100))
show(merged, cmap='viridis', clim=(0, 100))

In [ ]:
# convert every image to .tif
from os.path import exists
from pathlib import Path
import glob
root_folder = '/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/VIIRS_SCA'
# folders = [ f.path for f in os.scandir(root_folder) if f.is_dir() ]
h5paths = glob.iglob(root_folder+"/**/*.h5")
errors = []
skipped = []
for path in h5paths:
    path = Path(path)
    tif = str(path.parent.joinpath(path.stem)) + ".tif"
    if exists(tif):
        skipped.append(tif)
        continue
    try:
        # print(path)
        convert_h5_to_geotiff(path, datafield_name=datafield, output_geotiff=tif)
    except KeyError as e:
        errors.append(str(path.stem))
    except Exception as e:
        print(tif, e)


In [ ]:
errors.sort()
errors

In [ ]:
errors